In [1]:
from arraytainers import Numpytainer, Jaxtainer
import numpy as np
from numpy import array
import jax.numpy as jnp
import functools
import jaxlib

In [35]:
# a = [array([[3.74540119, 9.50714306, 7.31993942],
#        [5.98658484, 1.5601864 , 1.5599452 ],
#        [0.58083612, 8.66176146, 6.01115012]]), array([[3.74540119, 9.50714306],
#        [7.31993942, 5.98658484]])]
# key = array([[ 0, -1],[ 0, -1]])
a = [jnp.array([[3.74540119, 9.50714306, 7.31993942],
       [5.98658484, 1.5601864 , 1.5599452 ],
       [0.58083612, 8.66176146, 6.01115012]]), jnp.array([[3.74540119, 9.50714306],
       [7.31993942, 5.98658484]])]
key = jnp.array([10])

In [36]:
a[0][key]

DeviceArray([[0.5808361, 8.661761 , 6.01115  ]], dtype=float32)

In [17]:
A = Jaxtainer(a)
B = Numpytainer(a)

In [18]:
C = B[key]

[[3.74540119 9.50714306 7.31993942]
 [5.98658484 1.5601864  1.5599452 ]
 [0.58083612 8.66176146 6.01115012]] <class 'numpy.ndarray'>
[[10 -1]
 [ 0 -1]] <class 'numpy.ndarray'>


IndexError: index 10 is out of bounds for axis 0 with size 3

In [19]:
C = A[key]

[[3.7454011 9.507143  7.3199396]
 [5.9865847 1.5601864 1.5599452]
 [0.5808361 8.661761  6.01115  ]] <class 'jaxlib.xla_extension.DeviceArray'>
[[10 -1]
 [ 0 -1]] <class 'jaxlib.xla_extension.DeviceArray'>
[[[0.5808361 8.661761  6.01115  ]
  [0.5808361 8.661761  6.01115  ]]

 [[3.7454011 9.507143  7.3199396]
  [0.5808361 8.661761  6.01115  ]]]
[[3.7454011 9.507143 ]
 [7.3199396 5.9865847]] <class 'jaxlib.xla_extension.DeviceArray'>
[[10 -1]
 [ 0 -1]] <class 'jaxlib.xla_extension.DeviceArray'>
[[[7.3199396 5.9865847]
  [7.3199396 5.9865847]]

 [[3.7454011 9.507143 ]
  [7.3199396 5.9865847]]]


In [6]:
jnp.array([3.74540119, 9.50714306, 7.31993942])[jnp.array([ 0, -1])]

DeviceArray([3.7454011, 7.3199396], dtype=float32)

In [7]:
index_func = lambda contents : contents[key]
apply_func_to_contents(a, func=index_func, args=(key,))

NameError: name 'apply_func_to_contents' is not defined

In [8]:
def apply_func_to_contents(*contents, func=None, args=(), kwargs=None, index_args=None, 
                            return_list=False, throw_exception=False):

    try:
        expected = call_func_recursive(contents, func, args, kwargs, index_args, return_list)
        exception = None
    except Exception as error:
        if throw_exception:
            raise error
        else:
            expected = None
            exception = error

    return (expected, exception)

def call_func_recursive(contents, func, args, kwargs, index_args, return_list):
    
    first_item = contents[0]
    
    if kwargs is None:
      kwargs = {}

    if index_args is None:
        index_args = lambda args, idx : args

    if isinstance(first_item, dict):
        contents_i = {key: [item_i[key] for item_i in contents] for key in first_item.keys()}
        result = {key: call_func_recursive(contents_i[key], func, index_args(args,key), kwargs, index_args, return_list) 
                  for key in first_item.keys()}
        return list(result.values()) if return_list else result

    elif isinstance(first_item, (tuple,list)):
        contents_i = [[item_i[idx] for item_i in contents] for idx in range(len(first_item))]
        return [call_func_recursive(contents_i[idx], func, index_args(args,idx), kwargs, index_args, return_list)  
                for idx in range(len(first_item))]

    else:
        print(contents, args)
        return func(*contents, *args, **kwargs)